In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    WELFAKE
'''



data = pd.read_csv(r"../data/Des_fake_news/WELFAKE_PROCESSED.csv")
data["flag"] = data["label"]
data = data.sample(100)
data



,Unnamed: 0.1,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_comp,title_vader_neg,title_vader_neu,title_vader_pos,text_vader_neg,text_vader_neu,text_vader_pos,flag
3190,30617,30617,Ex New York Officer Gets Years of Probation i...,Former Officer Peter Liang will not serve any ...,0,0.012942,0.412762,0.136364,0.454545,-0.5423,0.189,0.811,0.000,0.153,0.723,0.124,0
1872,34112,34112,MarkLevin is Freaking Awesome Obama negotiate...,Let s get real with some awesome truth from Ma...,1,0.450000,0.433333,0.525000,0.750000,-0.3400,0.275,0.551,0.174,0.092,0.723,0.185,1
3931,41186,41186,South Korean President Moon to visit China Dec...,BEIJING Reuters South Korean President Moo...,0,-0.058333,0.341667,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.094,0.886,0.020,0
2378,63751,63751,Is Westworld HBOs Next Big Hit The New York...,On the morning they finished shooting the pilo...,0,0.166959,0.466318,0.045455,0.184848,0.0000,0.000,1.000,0.000,0.057,0.816,0.127,0
2113,27516,27516,TRUMP ADVISOR Has Warning For Syria That Has S...,Deputy Assistant to the President Sebastian Go...,1,-0.048214,0.310714,0.000000,0.000000,0.1027,0.169,0.634,0.197,0.170,0.761,0.069,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,25089,25089,Donald Trump Sends Executive Order Lawsuit to ...,The Department of Justice is formally objectin...,0,0.084398,0.481746,0.250000,0.500000,-0.2263,0.174,0.826,0.000,0.105,0.810,0.085,0
4751,12343,12343,WATCH General BOMBARDS Trump For The Most An...,As you may already know Trump recently shocke...,1,0.059167,0.491333,0.183333,0.333333,-0.3804,0.147,0.853,0.000,0.101,0.802,0.097,1
3877,13639,13639,Putin and Macron discuss North Korea s missile...,MOSCOW Reuters Russian President Vladimir ...,0,0.150000,0.325000,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.092,0.849,0.059,0
1000,63136,63136,Comment on Hillary Clinton is an alcoholic by ...,The American Medical Association defines an al...,1,-0.013639,0.422944,-0.250000,0.500000,0.0000,0.000,1.000,0.000,0.133,0.790,0.076,1


In [2]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd

def tf_idf_vectorize(df: pd.DataFrame, corpus: pd.Series, vocabulary: list[str]) -> tuple[list[str], pd.DataFrame]:
    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3), vocabulary=vocabulary)
    features = vectorizer.fit_transform(corpus).toarray()
    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(features, columns=headers)
    #final = pd.concat([df, feature_frame], axis=1)
    return (names, feature_frame)


def get_full_vocabulary(corpus: pd.Series) -> list[str]:
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stopwords.words("english"), max_features=500, ngram_range=(1,3))
    vectorizer.fit_transform(corpus).toarray()
    return vectorizer.get_feature_names_out()


def add_tf_idf_vector(x: pd.DataFrame, partial_corpus: pd.Series, full_corpus: pd.Series) -> pd.DataFrame:
    temp = pd.concat((x, partial_corpus), axis=1).reset_index()
    # first, we need a vocabulary from the entire dataset
    vocabulary = get_full_vocabulary(full_corpus)

    tf_idf_vector = tf_idf_vectorize(temp, temp[partial_corpus.name], vocabulary=vocabulary)[1].reset_index()
    temp = temp.reset_index()
    temp = pd.concat((temp, tf_idf_vector), axis=1).drop([partial_corpus.name, "index"], axis=1)
    return temp



In [3]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"]

filtered["corpus"] = filtered.apply(lambda x: " ".join([x["title"], x["text"]]), axis=1)


# raw polarity and subjectivity scores from Textblob, Vader
features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_comp",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]]

# dataframe to store accuracies for NN and log regression
#accuracy_df = pd.DataFrame(columns=["polarity",
#               "tfidf",
#               "combined"])


EXCEL_FILE = r"../data/Des_fake_news/Sentiment_Analysis_Results/WELFAKE_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


In [4]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io

'''
    Get ROC plot ready
'''
plt.figure()
lw = 2

<Figure size 640x480 with 0 Axes>

In [ ]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io



# average accuracies
comb_output = {"train" : None, "test" : None}
comb_matrices = []


comb_fpr = []
comb_tpr = []
comb_thresh = []

comb_auc = 0


train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index], filtered["corpus"])
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], filtered["corpus"].iloc[test_index], filtered["corpus"])
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    comb_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    comb_ensemble.fit(x_train, y_train)
    y_pred = comb_ensemble.predict(x_test)

    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    comb_fpr.append(fpr)
    comb_tpr.append(tpr)
    comb_thresh.append(thresh)
    comb_auc += metrics.auc(fpr, fpr)

    comb_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = comb_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

comb_output["test"] = [(test_total / run_count)]
comb_output["train"] = [(train_total / run_count)]
comb_output = pd.DataFrame(comb_output)

print("Average confusion matrix:")
comb_matrices = np.mean(np.array(comb_matrices), axis=0)
print(comb_matrices)


comb_fpr = np.mean(np.array(comb_fpr), axis=0)
comb_tpr = np.mean(np.array(comb_tpr), axis=0)
comb_thresh = np.mean(np.array(comb_thresh), axis=0)

comb_auc = comb_auc / float(run_count)


plt.plot(comb_fpr, comb_tpr,
 lw=lw, label='Polarity + Tf-Idf  (%0.2f)' % comb_auc)

# write confusion matrix and save
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book


pd.DataFrame(comb_matrices).to_excel(writer, sheet_name=f"combined_matrix")

book.save(filename=EXCEL_FILE)
book.close()









comb_output

0
tf_idf training
tf_idf testing
ensemble fitting


ValueError: y should be a 1d array, got an array of shape (1, 3) instead.

In [ ]:
'''
    POLARITY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
pol_output = {"train" : None, "test" : None}
pol_matrices = []


pol_fpr = []
pol_tpr = []
pol_thresh = []

pol_auc = 0

train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
  
    x_train = x.iloc[train_index]
    x_train = scaler.fit_transform(x_train)

    x_test = x.iloc[test_index]
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    pol_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    pol_ensemble.fit(x_train, y_train)
    y_pred = pol_ensemble.predict(x_test)

    pol_fpr.append(fpr)
    pol_tpr.append(tpr)
    pol_thresh.append(thresh)
    pol_auc += metrics.auc(fpr, tpr)

    pol_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = pol_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

pol_output["test"] = [(test_total / run_count)]
pol_output["train"] = [(train_total / run_count)]
pol_output = pd.DataFrame(pol_output)

print("Average confusion matrix:")
pol_matrices = np.mean(np.array(pol_matrices), axis=0)
print(pol_matrices)

pol_fpr = np.mean(np.array(pol_fpr), axis=0)
pol_tpr = np.mean(np.array(pol_tpr), axis=0)
pol_thresh = np.mean(np.array(pol_thresh), axis=0)

pol_auc = pol_auc / float(run_count)


plt.plot(pol_fpr, pol_tpr,
 lw=lw, label='Polarity  (%0.2f)' % pol_auc)

# write confusion matrix and save
book = load_workbook(EXCEL_FILE)
riter = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book


pd.DataFrame(pol_matrices).to_excel(writer, sheet_name=f"polarity_matrix")

book.save(filename=EXCEL_FILE)
book.close()


pol_output

In [ ]:
'''
    TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
tf_output = {"train" : None, "test" : None}
tf_matrices = []

train_total = 0
test_total = 0
run_count = 0

tf_fpr = []
tf_tpr = []
tf_thresh = []

tf_auc = 0

#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index], filtered["corpus"]).drop(features, axis=1)
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], filtered["corpus"].iloc[test_index], filtered["corpus"]).drop(features, axis=1)
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    tf_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    tf_ensemble.fit(x_train, y_train)
    y_pred = tf_ensemble.predict(x_test)

    tf_fpr.append(fpr)
    tf_tpr.append(tpr)
    tf_thresh.append(thresh)
    tf_auc += metrics.auc(fpr, tpr)


    tf_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = tf_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

tf_output["test"] = [(test_total / run_count)]
tf_output["train"] = [(train_total / run_count)]
tf_output = pd.DataFrame(tf_output)

print("Average confusion matrix:")
tf_matrices = np.mean(np.array(tf_matrices), axis=0)
print(tf_matrices)

tf_fpr = np.mean(np.array(tf_fpr), axis=0)
tf_tpr = np.mean(np.array(tf_tpr), axis=0)
tf_thresh = np.mean(np.array(tf_thresh), axis=0)

tf_auc = tf_auc / float(run_count)


plt.plot(tf_fpr, tf_tpr,
 lw=lw, label='Tf-Idf  (%0.2f)' % tf_auc)

# write confusion matrix and save
book = load_workbook(EXCEL_FILE)
riter = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book


# write confusion matrix and save
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book




pd.DataFrame(comb_matrices).to_excel(writer, sheet_name=f"tfidf_matrix")

book.save(filename=EXCEL_FILE)
book.close()


tf_output

In [ ]:
'''
    combine accuracies into one table
'''

final_results = pd.concat((pol_output, tf_output, comb_output), axis=0)
final_results.index = ["Polarity", "Tf-Idf", "Polarity + Tf-Idf"]
final_results

In [ ]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''




book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
final_results.to_excel(writer, sheet_name=f"accuracies")
book.save(filename=EXCEL_FILE)
book.close()



In [ ]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io


raw_fpr, raw_tpr, raw_thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
raw_roc_auc = metrics.auc(raw_fpr, raw_tpr)


In [ ]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io




plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()



In [ ]:
unique, counts = np.unique(y_test, return_counts=True)
counts[0] / sum(counts)